In [1]:

import numpy as np
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50
from matplotlib import pyplot as plt
import tensorflow_datasets as tfds

print("Tensorflow version " + tf.__version__)

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.
Tensorflow version 2.15.0


In [2]:
BATCH_SIZE = 32
classes = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

preprocessing image first converting them into float 32
then the resnet preprocess function the ime acccording to the resnte 50 archittecture

In [3]:
def preprocessimage(input):
  input_image=input.astype('float32')
  output_image=tf.keras.applications.resnet50.preprocess_input(input_image)
  return output_image




In [4]:
#train image and test image loading Cifar10.data set then preprocessing the image
(train_dataset,train_labels),(val_dataset,val_labels)=tf.keras.datasets.cifar10.load_data()
trainx=preprocessimage(train_dataset)
valx=preprocessimage(val_dataset)

170498071/170498071 [==============================] - 4s 0us/step


In [5]:
#isme ham feature extractor layer from resnte utha rhay and then ham uspe transfer elarning apply kre ge
def feature_extractor(inputs):

  feature_extractor = tf.keras.applications.resnet.ResNet50(input_shape=(224, 224, 3),
                                               include_top=False,
                                               weights='imagenet')(inputs)
  return feature_extractor

def classifier(inputs):
    x = tf.keras.layers.GlobalAveragePooling2D()(inputs)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(1024, activation="relu")(x)
    x = tf.keras.layers.Dense(512, activation="relu")(x)
    x = tf.keras.layers.Dense(10, activation="softmax", name="classification")(x)
    return x

#isme ham 32 32 3 image ko up sample kre ge by 7 7 take woh 224 224 hojaye or resnet ki feature ectractor layer me sahi chali jaye
def final_model(inputs):

    resize = tf.keras.layers.UpSampling2D(size=(7,7))(inputs)

    resnet_feature_extractor = feature_extractor(resize)
    classification_output = classifier(resnet_feature_extractor)

    return classification_output

def define_compile_model():
  inputs = tf.keras.layers.Input(shape=(32,32,3))

  classification_output = final_model(inputs)
  model = tf.keras.Model(inputs=inputs, outputs = classification_output)

  model.compile(optimizer='SGD',
                loss='sparse_categorical_crossentropy',
                metrics = ['accuracy'])

  return model



In [6]:
model = define_compile_model()

model.summary()

94765736/94765736 [==============================] - 1s 0us/step
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 up_sampling2d (UpSampling2  (None, 224, 224, 3)       0         
 D)                                                              
                                                                 
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 global_average_pooling2d (  (None, 2048)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 flatten (Flatten)           (None, 2048)              0         
                                                              

In [7]:
history = model.fit(trainx, train_labels, epochs=4, validation_data = (valx, val_labels), batch_size=64)


Epoch 1/4
782/782 [==============================] - 578s 705ms/step - loss: 0.4021 - accuracy: 0.8692 - val_loss: 0.1899 - val_accuracy: 0.9331
Epoch 2/4
130/782 [===>..........................] - ETA: 7:04 - loss: 0.1091 - accuracy: 0.9666

KeyboardInterrupt: 